In [3]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
files = os.listdir('data/brenda')
files

['result_inhibitors.csv',
 'result_inhibitors_filtrado.csv',
 'result_kcat_km.csv',
 'result_kcat_km_filtrado .csv',
 'result_ki+filtrado .csv',
 'result_ki.csv',
 'result_turnover.csv',
 'result_turnover_filtrado.csv']

### Descrição
* result* : bruto
* result*filtrado : so AMPC

* inhibitors: moléculas relacionadas ao EcNumber 3.5.2.6
* result_ki(contante de inibição) + result_turnover(Kcat) = Kcat_Km : JOIN
* result_kcat_km: mostra o kcat_km de EcNumber 3.5.2.6

## NaN e Null

In [9]:
data_files = {k[:-4]: pd.read_csv(f'data/brenda/{v}', sep=';') for k,v in zip(files,files)}
#data_files

In [38]:
cols = []
for k, data in data_files.items():
    print(k.upper())
    print('---isnull---\n', data.isnull().sum(), end='\n\n')
    print('---NaN---\n', data.isna().sum(), end='\n\n')
    cols.append(data.columns)
    print(40*'-', end='\n\n')

RESULT_INHIBITORS
---isnull---
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           405
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLASS                           27
dtype: int64

---NaN---
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           405
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLASS                           27
dtype: int64

----------------------------------------

RESULT_INHIBITORS_FILTRADO
---isnull---
 Unnamed: 0                       0
literature inhibitor             0
inhibitor                        0
commentary inhibitor           279
organism                         0
ecNumber                         0
ligandStructureId inhibitor      0
CLASS                            0
dtype:

In [40]:
for file, df in data_files.items():
    print(file, '---', df.shape[0]) #não se complementam

result_inhibitors --- 692
result_inhibitors_filtrado --- 465
result_kcat_km --- 392
result_kcat_km_filtrado  --- 244
result_ki+filtrado  --- 197
result_ki --- 292
result_turnover --- 1388
result_turnover_filtrado --- 961


## Feature por file

In [52]:
listas_normalizadas = [sorted([col.lower() for col in df.columns]) for df in data_files.values()]

iguais = all(sublista == listas_normalizadas[0] for sublista in listas_normalizadas)

col_file = pd.concat([pd.Series(cols) for cols in listas_normalizadas], axis=1)
col_file.columns = [nome_arquivo for nome_arquivo in data_files.keys()]

col_file

,result_inhibitors,result_inhibitors_filtrado,result_kcat_km,result_kcat_km_filtrado,result_ki+filtrado,result_ki,result_turnover,result_turnover_filtrado
0,class,class,class,class,class,class,class,class
1,commentary inhibitor,commentary inhibitor,commentary kcat/km,commentary kcat/km,commentary ki,commentary ki,commentary turnover,commentary turnover
2,ecnumber,ecnumber,kcatkmvalue,kcatkmvalue,inhibitor ki,inhibitor ki,ligandstructureid turnover,ligandstructureid turnover
3,inhibitor,inhibitor,kcatkmvaluemaximum,kcatkmvaluemaximum,kivalue,kivalue,literature turnover,literature turnover
4,ligandstructureid inhibitor,ligandstructureid inhibitor,ligandstructureid kcat/km,ligandstructureid kcat/km,kivaluemaximum,kivaluemaximum,organism,organism
5,literature inhibitor,literature inhibitor,literature kcat/km,literature kcat/km,ligandstructureid ki,ligandstructureid ki,substrate turnover,substrate turnover
6,organism,organism,organism,organism,literature ki,literature ki,turnovernumber,turnovernumber
7,unnamed: 0,unnamed: 0,substrate,substrate,organism,organism,turnovernumbermaximum,turnovernumbermaximum


In [58]:
col_file[['result_inhibitors','result_kcat_km','result_ki','result_turnover']]

,result_inhibitors,result_kcat_km,result_ki,result_turnover
0,class,class,class,class
1,commentary inhibitor,commentary kcat/km,commentary ki,commentary turnover
2,ecnumber,kcatkmvalue,inhibitor ki,ligandstructureid turnover
3,inhibitor,kcatkmvaluemaximum,kivalue,literature turnover
4,ligandstructureid inhibitor,ligandstructureid kcat/km,kivaluemaximum,organism
5,literature inhibitor,literature kcat/km,ligandstructureid ki,substrate turnover
6,organism,organism,literature ki,turnovernumber
7,unnamed: 0,substrate,organism,turnovernumbermaximum


## A fazer
### 3 bases
- AMPCs
- todos
- sem AMPCs

### Features
- inhibitors
- kcat_km
- ki
- turnover

### Falta
- ID
- sequences - enzimas
- smiles - substrato

In [128]:
i = pd.concat([data_files[l[1]]['organism'] + '__' + data_files[l[1]]['substrate'].sort_values(),
               data_files[l[2]]['organism'] + '__' + data_files[l[1]]['substrate'].sort_values(),
              data_files[l[3]]['organism'] + '__' + data_files[l[1]]['substrate'].sort_values()],
              axis=1)
i.columns = [l[1], l[2], l[3]]

In [168]:
count = 0
a = set(i[l[1]].dropna())
b = set(i[l[2]].dropna())
c = set(i[l[3]].dropna())

iguais_nos_tres = a & b & c

print(len(iguais_nos_tres))

9


In [169]:
# todos = pd.concat([a, b, c], ignore_index=True)
# duplicados = todos.value_counts()
# iguais = duplicados[duplicados > 1]

# print(duplicados)